## For Rakuten Travel

In [52]:
import time
import random
import json
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException,\
    TimeoutException, NoSuchElementException, StaleElementReferenceException

## Hotel Test

## single page scrape Test

In [ ]:
s=Service(ChromeService(ChromeDriverManager().install()))
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')

driver = webdriver.Chrome(service=s, options=options)
# driver.maximize_window()
driver.delete_all_cookies()
driver.get('https://kw.travel.rakuten.co.jp/keyword/Event.do?f_teikei=groupkw&f_query=&f_max=30&f_area=&f_chu=&f_shou=&f_category=0&f_sort=0&f_flg=&f_cd_application=&f_cd_chain=&f_all_chain=1&f_su=2&f_invoice_qualified=0&f_next=1')

# Parse page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [40]:
# Locate the main section that contains all hotel details
main_section = soup.find('div', id='result')

if main_section:
    # Loop through all elements with class 'hotelBox'
    hotel_boxes = main_section.find_all('div', class_='hotelBox')
    for hotel_box in hotel_boxes:
        # Extract hotel name
        hotel_name_element = hotel_box.select_one('.hotelName')
        hotel_name = hotel_name_element.get_text(strip=True) if hotel_name_element else 'N/A'
        print(f"Hotel Name: {hotel_name}")

        # Extract hotel rating
        rating_imgs = hotel_box.select('.hotelRating img[alt]')
        ratings = [img['alt'] for img in rating_imgs]
        print(f"Hotel Rating: {' '.join(ratings)}")

        # Extract hotel price details
        price_details_element = hotel_box.select_one('.price dd')
        price_details = price_details_element.get_text(strip=True) if price_details_element else 'N/A'
        print(f"Price Details: {price_details}")

        # Extract plan details
        plans = hotel_box.select('.planBox')
        for plan in plans:
            plan_title_element = plan.select_one('h3')
            plan_title = plan_title_element.get_text(strip=True) if plan_title_element else 'N/A'
            
            plan_description_element = plan.select_one('.planOutline')
            plan_description = plan_description_element.get_text(strip=True) if plan_description_element else 'N/A'
            
            # Extract plan prices
            plan_prices = plan.select('.rmTypPrc dd.plnPrc')
            for price in plan_prices:
                print(f"Plan Price: {price.get_text(strip=True)}")
                
            print(f"Plan Title: {plan_title}")
            print(f"Plan Description: {plan_description}")
        print('-' * 40)
else:
    print("Main section with id 'result' not found.")

Hotel Name: コンフォートイン鳥栖(旧チサンイン鳥栖)
Hotel Rating: 総合 0 1 2 3 4
Price Details: 2,455円～（消費税込2,700円～）
Plan Price: 4,546円～13,000円/人（消費税込5,000円～14,300円/人）
Plan Title: 【30連泊割】鳥栖ICより車で約6分◆青空駐車場無料◆オフィス利用や長期滞在に◆素泊まり◆
Plan Description: 30泊以上の連泊に対応した特別割引プランです。
30泊未満に変更の場合、返金は致しかねますので予めご了承ください。

長期出張・単身赴任でお部屋を探している方、簡易オフィスとしてご利用されたい方へおすすめのプランです。
...
Plan Price: 5,819円～10,364円/人（消費税込6,400円～11,400円/人）
Plan Title: 【21日前早期割】4連泊以上でお得◆エコステイ◆鳥栖ICより車で約6分◆朝食付◆
Plan Description: ■21日前まで×4連泊以上でおトク■
4連泊以上の連泊宿泊に対応した特別割引プランです。

【注意事項】
本プランは、清掃無しのプランです。
滞在時の客室清掃が必要な場合、別途料金「清掃代　1，000円（税込）」を頂戴い...
Plan Price: 4,637円～9,273円/人（消費税込5,100円～10,200円/人）
Plan Title: 【開業記念15％OFF】鳥栖ICより車で約6分◆青空駐車場無料◆素泊まり◆
Plan Description: ■期間限定■開業記念特別プラン

最新のコンフォートブランドをオトクに体験していただけるプランをご用意！

※本プランは朝食付きではありません。

●コンフォートイン鳥栖のこだわり

〈停めやすい無料駐車場〉
広々とした...
----------------------------------------
Hotel Name: ＨＯＴＥＬ　ＡＺ　佐賀吉野ヶ里店
Hotel Rating: 総合 0 1 2 3 4
Price Details: 3,500円～（消費税込3,850円～）
Plan Price: 3,500円/人（消費税込3,850円/人）
Plan Title: 【朝食無料】スタンダードプラン

## Multiple page scraping

In [54]:
def setup_driver(url):
    s = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    
    driver = webdriver.Chrome(options=options, service=s)
    driver.maximize_window()
    driver.delete_all_cookies()
    driver.get(url)
    time.sleep(2)  # Wait for the page to load
    
    return driver

In [55]:
# Get the last page number
test = setup_driver('https://kw.travel.rakuten.co.jp/keyword/Event.do?f_teikei=groupkw&f_query=&f_max=30&f_area=&f_chu=&f_shou=&f_category=0&f_sort=0&f_flg=&f_cd_application=&f_cd_chain=&f_all_chain=1&f_su=2&f_invoice_qualified=0&f_next=1')
last_page_element = test.find_element(By.XPATH, '//*[@id="result"]/div[1]/ul/li[1]/a[5]')
last_page_number = int(last_page_element.text.strip())
last_page_number

184

#### (Replace the middle number with last_page_number number to  scrape all page)

In [56]:
# Prepare to store data
data = []

# Loop through each page 
for page_number in range(1, 2 + 1):
    # Load the page
    url = f'https://kw.travel.rakuten.co.jp/keyword/Event.do?f_teikei=groupkw&f_query=&f_max=30&f_area=&f_chu=&f_shou=&f_category=0&f_sort=0&f_flg=&f_cd_application=&f_cd_chain=&f_all_chain=1&f_su=2&f_invoice_qualified=0&f_next={page_number}'
    
    s=Service(ChromeDriverManager().install()) 
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=options,service=s)
    driver.maximize_window()
    driver.delete_all_cookies()
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Locate the main section that contains all hotel details
    main_section = soup.find('div', id='result')

    if main_section:
        # Loop through all elements with class 'hotelBox'
        hotel_boxes = main_section.find_all('div', class_='hotelBox')
        for hotel_box in hotel_boxes:
            # Extract hotel name
            hotel_name_element = hotel_box.select_one('.hotelName')
            hotel_name = hotel_name_element.get_text(strip=True) if hotel_name_element else 'N/A'
            
            # Extract hotel address
            hotel_address_element = hotel_box.select_one('.hotelOutline .city')
            hotel_address = hotel_address_element.get_text(strip=True) if hotel_name_element else 'N/A'
            
            # Extract hotel price details
            price_details_element = hotel_box.select_one('.price dd')
            price_details = price_details_element.get_text(strip=True) if price_details_element else 'N/A'
            
            # Extract plan details
            plans = hotel_box.select('.planBox')
            for plan in plans:
                plan_title_element = plan.select_one('h3')
                plan_title = plan_title_element.get_text(strip=True) if plan_title_element else 'N/A'
                
                plan_description_element = plan.select_one('.planOutline')
                plan_description = plan_description_element.get_text(strip=True) if plan_description_element else 'N/A'
                
                # Extract plan prices
                plan_prices = plan.select('.rmTypPrc dd.plnPrc')
                plan_prices_text = [price.get_text(strip=True) for price in plan_prices]
                
                # Append data
                data.append({
                    'Hotel Name': hotel_name,
                    'Hotel Address': hotel_address,
                    'Price Details': price_details,
                    'Plan Title': plan_title,
                    'Plan Description': plan_description,
                    'Plan Prices': '; '.join(plan_prices_text)
                })
    else:
        print(f"Main section with id 'result' not found on page {page_number}.")

# Close the WebDriver
driver.quit()

# Save data to CSV
df = pd.DataFrame(data)
# df.to_csv('hotel_data.csv', index=False, encoding='utf-8')

In [57]:
df

,Hotel Name,Hotel Address,Price Details,Plan Title,Plan Description,Plan Prices
0,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【30連泊割】鳥栖ICより車で約6分◆青空駐車場無料◆オフィス利用や長期滞在に◆素泊まり◆,30泊以上の連泊に対応した特別割引プランです。\n30泊未満に変更の場合、返金は致しかねます...,"4,546円～13,000円/人（消費税込5,000円～14,300円/人）"
1,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【21日前早期割】4連泊以上でお得◆エコステイ◆鳥栖ICより車で約6分◆朝食付◆,■21日前まで×4連泊以上でおトク■\n4連泊以上の連泊宿泊に対応した特別割引プランです。\...,"5,819円～10,364円/人（消費税込6,400円～11,400円/人）"
2,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【開業記念15％OFF】鳥栖ICより車で約6分◆青空駐車場無料◆素泊まり◆,■期間限定■開業記念特別プラン\n\n最新のコンフォートブランドをオトクに体験していただける...,"4,637円～9,273円/人（消費税込5,100円～10,200円/人）"
3,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【朝食無料】スタンダードプラン,HOTEL AZ のスタンダードなプランです。\n【朝食】 6：00～10：00（最終入場 ...,"3,500円/人（消費税込3,850円/人）"
4,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【夕食付き】スタンダードプラン ※朝食無料,当館人気の夕食バイキングが付いたプランです。\n【夕食】18：00～22：00（最終入場21...,"4,373円/人（消費税込4,810円/人）"
...,...,...,...,...,...,...
170,ＨＯＴＥＬ Ｒ９ Ｔｈｅ Ｙａｒｄ 甲賀,滋賀県 信楽・甲賀,"3,546円～（消費税込3,900円～）",【スタンダード】ビジネスや観光の拠点に♪軽食＆ホットドリンク付【駐車場至近＆無料】,《シンプルステイ★プランに迷ったらこれ！》\nビジネス＆観光の拠点にも最適♪\n\n◆◆ご連...,"3,728円～7,637円/人（消費税込4,100円～8,400円/人）"
171,ＨＯＴＥＬ Ｒ９ Ｔｈｅ Ｙａｒｄ 甲賀,滋賀県 信楽・甲賀,"3,546円～（消費税込3,900円～）",【連泊プラン（フル清掃）】連泊におすすめ！毎日フル清掃★軽食＆ホットドリンク付【駐車場至近＆無料】,《3連泊以上のご滞在におすすめ！毎日フル清掃プラン♪》\nビジネスやレジャーに！プライベート...,"3,910円～7,819円/人（消費税込4,300円～8,600円/人）"
172,ホテルルートイン甲賀水口 －国道１号－,滋賀県 信楽・甲賀,"4,455円～（消費税込4,900円～）",早割10日前プラン【WOWOW視聴可能・バイキング朝食・Wi-Fi完備・大浴場完備・駐車場無料】,◎◎＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝◎◎\n10日前までにご予約のお客様に適用さ...,"4,705円～9,714円/人（消費税込5,175円～10,685円/人）"
173,ホテルルートイン甲賀水口 －国道１号－,滋賀県 信楽・甲賀,"4,455円～（消費税込4,900円～）",ルートイングループ共通お食事券1000円付プラン【WOWOW視聴可能・バイキング朝食・駐車場無料】,1泊1名様に付きルートイングループ共通お食事券（1000円）が付くプランです\n出張先のホテ...,"5,864円～8,864円/人（消費税込6,450円～9,750円/人）"


In [62]:
import json
from collections import defaultdict

# Assuming your data is in a variable called 'data'

def process_data(data):
    hotels = defaultdict(lambda: defaultdict(list))

    for item in data:
        hotel_name = item['Hotel Name']
        hotel_address = item['Hotel Address']
        price_details = item['Price Details']

        # Create or update hotel info
        hotels[hotel_name]['Hotel Address'] = hotel_address
        hotels[hotel_name]['Price Details'] = price_details

        # Add plan details
        plan = {
            'Plan Title': item['Plan Title'],
            'Plan Description': item['Plan Description'],
            'Plan Prices': item['Plan Prices']
        }
        hotels[hotel_name]['Plans'].append(plan)

    return dict(hotels)

# Process the data
processed_data = process_data(data)

print(json.dumps(processed_data, ensure_ascii=False, indent=4))

{
    "コンフォートイン鳥栖(旧チサンイン鳥栖)": {
        "Hotel Address": "佐賀県 鳥栖",
        "Price Details": "2,455円～（消費税込2,700円～）",
        "Plans": [
            {
                "Plan Title": "【30連泊割】鳥栖ICより車で約6分◆青空駐車場無料◆オフィス利用や長期滞在に◆素泊まり◆",
                "Plan Description": "30泊以上の連泊に対応した特別割引プランです。\n30泊未満に変更の場合、返金は致しかねますので予めご了承ください。\n\n長期出張・単身赴任でお部屋を探している方、簡易オフィスとしてご利用されたい方へおすすめのプランです。\n...",
                "Plan Prices": "4,546円～13,000円/人（消費税込5,000円～14,300円/人）"
            },
            {
                "Plan Title": "【21日前早期割】4連泊以上でお得◆エコステイ◆鳥栖ICより車で約6分◆朝食付◆",
                "Plan Description": "■21日前まで×4連泊以上でおトク■\n4連泊以上の連泊宿泊に対応した特別割引プランです。\n\n【注意事項】\n本プランは、清掃無しのプランです。\n滞在時の客室清掃が必要な場合、別途料金「清掃代　1，000円（税込）」を頂戴い...",
                "Plan Prices": "5,819円～10,364円/人（消費税込6,400円～11,400円/人）"
            },
            {
                "Plan Title": "【開業記念15％OFF】鳥栖ICより車で約6分◆青空駐車場無料◆素泊まり◆",
                "Plan Description": "■期間限定■開業記念特別プラン\n\n最新のコンフォートブランドをオトクに体験していただけるプランをご用意！\n\n※本プランは朝食付きではありま

### Process Data

In [2]:
import pandas as pd
df = pd.read_csv(r"./scraped_data/hotel_data_20240822_225913.csv")
df.head()

,Hotel Name,Hotel Address,Price Details,Plan Title,Plan Description,Plan Prices
0,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【30連泊割】鳥栖ICより車で約6分◆青空駐車場無料◆オフィス利用や長期滞在に◆素泊まり◆,30泊以上の連泊に対応した特別割引プランです。\n30泊未満に変更の場合、返金は致しかねます...,"4,546円～13,000円/人（消費税込5,000円～14,300円/人）"
1,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【21日前早期割】4連泊以上でお得◆エコステイ◆鳥栖ICより車で約6分◆朝食付◆,■21日前まで×4連泊以上でおトク■\n4連泊以上の連泊宿泊に対応した特別割引プランです。\...,"5,819円～10,364円/人（消費税込6,400円～11,400円/人）"
2,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【開業記念15％OFF】鳥栖ICより車で約6分◆青空駐車場無料◆素泊まり◆,■期間限定■開業記念特別プラン\n\n最新のコンフォートブランドをオトクに体験していただける...,"4,637円～9,273円/人（消費税込5,100円～10,200円/人）"
3,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【朝食無料】スタンダードプラン,HOTEL AZ のスタンダードなプランです。\n【朝食】 6：00～10：00（最終入場 ...,"3,500円/人（消費税込3,850円/人）"
4,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【夕食付き】スタンダードプラン ※朝食無料,当館人気の夕食バイキングが付いたプランです。\n【夕食】18：00～22：00（最終入場21...,"4,373円/人（消費税込4,810円/人）"


In [10]:
import re

def extract_prices(plan_prices):
    # Base price range: match prices like "4,546円～13,000円/人" or "5,819円/人"
    base_price_match = re.search(r'(\d{1,3}(?:,\d{3})?円(?:～\d{1,3}(?:,\d{3})?円)?/人)', plan_prices)
    
    # Tax-inclusive price range: match prices like "（消費税込5,000円～14,300円/人）"
    tax_price_match = re.search(r'（消費税込(\d{1,3}(?:,\d{3})?円(?:～\d{1,3}(?:,\d{3})?円)?/人)）', plan_prices)
    
    base_price = base_price_match.group(1) if base_price_match else None
    tax_price = tax_price_match.group(1) if tax_price_match else None
    
    return base_price, tax_price

In [7]:
df.head()

,Hotel Name,Hotel Address,Price Details,Plan Title,Plan Description,Plan Prices
0,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【30連泊割】鳥栖ICより車で約6分◆青空駐車場無料◆オフィス利用や長期滞在に◆素泊まり◆,30泊以上の連泊に対応した特別割引プランです。\n30泊未満に変更の場合、返金は致しかねます...,"4,546円～13,000円/人（消費税込5,000円～14,300円/人）"
1,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【21日前早期割】4連泊以上でお得◆エコステイ◆鳥栖ICより車で約6分◆朝食付◆,■21日前まで×4連泊以上でおトク■\n4連泊以上の連泊宿泊に対応した特別割引プランです。\...,"5,819円～10,364円/人（消費税込6,400円～11,400円/人）"
2,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【開業記念15％OFF】鳥栖ICより車で約6分◆青空駐車場無料◆素泊まり◆,■期間限定■開業記念特別プラン\n\n最新のコンフォートブランドをオトクに体験していただける...,"4,637円～9,273円/人（消費税込5,100円～10,200円/人）"
3,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【朝食無料】スタンダードプラン,HOTEL AZ のスタンダードなプランです。\n【朝食】 6：00～10：00（最終入場 ...,"3,500円/人（消費税込3,850円/人）"
4,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【夕食付き】スタンダードプラン ※朝食無料,当館人気の夕食バイキングが付いたプランです。\n【夕食】18：00～22：00（最終入場21...,"4,373円/人（消費税込4,810円/人）"


In [11]:
df['Plan Prices'].apply(extract_prices)

0     (4,546円～13,000円/人, 5,000円～14,300円/人)
1     (5,819円～10,364円/人, 6,400円～11,400円/人)
2      (4,637円～9,273円/人, 5,100円～10,200円/人)
3                     (3,500円/人, 3,850円/人)
4                     (4,373円/人, 4,810円/人)
                      ...                 
81      (5,871円～6,150円/人, 6,458円～6,765円/人)
82                    (4,678円/人, 5,145円/人)
83    (3,637円～11,819円/人, 4,000円～13,000円/人)
84    (3,091円～10,046円/人, 3,400円～11,050円/人)
85    (3,819円～12,410円/人, 4,200円～13,650円/人)
Name: Plan Prices, Length: 86, dtype: object

In [12]:
df['Base Price'], df['Tax Inclusive Price'] = zip(*df['Plan Prices'].apply(extract_prices))
df

,Hotel Name,Hotel Address,Price Details,Plan Title,Plan Description,Plan Prices,Base Price,Tax Inclusive Price
0,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【30連泊割】鳥栖ICより車で約6分◆青空駐車場無料◆オフィス利用や長期滞在に◆素泊まり◆,30泊以上の連泊に対応した特別割引プランです。\n30泊未満に変更の場合、返金は致しかねます...,"4,546円～13,000円/人（消費税込5,000円～14,300円/人）","4,546円～13,000円/人","5,000円～14,300円/人"
1,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【21日前早期割】4連泊以上でお得◆エコステイ◆鳥栖ICより車で約6分◆朝食付◆,■21日前まで×4連泊以上でおトク■\n4連泊以上の連泊宿泊に対応した特別割引プランです。\...,"5,819円～10,364円/人（消費税込6,400円～11,400円/人）","5,819円～10,364円/人","6,400円～11,400円/人"
2,コンフォートイン鳥栖(旧チサンイン鳥栖),佐賀県 鳥栖,"2,455円～（消費税込2,700円～）",【開業記念15％OFF】鳥栖ICより車で約6分◆青空駐車場無料◆素泊まり◆,■期間限定■開業記念特別プラン\n\n最新のコンフォートブランドをオトクに体験していただける...,"4,637円～9,273円/人（消費税込5,100円～10,200円/人）","4,637円～9,273円/人","5,100円～10,200円/人"
3,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【朝食無料】スタンダードプラン,HOTEL AZ のスタンダードなプランです。\n【朝食】 6：00～10：00（最終入場 ...,"3,500円/人（消費税込3,850円/人）","3,500円/人","3,850円/人"
4,ＨＯＴＥＬ ＡＺ 佐賀吉野ヶ里店,佐賀県 鳥栖,"3,500円～（消費税込3,850円～）",【夕食付き】スタンダードプラン ※朝食無料,当館人気の夕食バイキングが付いたプランです。\n【夕食】18：00～22：00（最終入場21...,"4,373円/人（消費税込4,810円/人）","4,373円/人","4,810円/人"
...,...,...,...,...,...,...,...,...
81,東横ＩＮＮ京王線橋本駅北口,神奈川県 大和・相模原・町田西部,"3,652円～（消費税込4,017円～）",★【無料の元気朝食付】いつでも安心価格♪ビジネス・レジャーに！ 禁煙デラックスツイン,ホテルチェーン東横インのスタンダードプランです\n\n「きれいで清潔なお部屋」\n「全店が同...,"5,871円～6,150円/人（消費税込6,458円～6,765円/人）","5,871円～6,150円/人","6,458円～6,765円/人"
82,東横ＩＮＮ京王線橋本駅北口,神奈川県 大和・相模原・町田西部,"3,652円～（消費税込4,017円～）",★【当日24時以降もご予約OK！】禁煙エコノミーツイン,当日深夜２４時以降も予約いただけるプランです\n\n★ 全室ユニットバス完備 ★ 朝食無料サ...,"4,678円/人（消費税込5,145円/人）","4,678円/人","5,145円/人"
83,ホテルリブマックスＢＵＤＧＥＴ相模原,神奈川県 大和・相模原・町田西部,"3,091円～（消費税込3,400円～）",【10時チェックアウト】スタンダードプラン！淵野辺駅北口徒歩2分,～最寄駅からホテルまで～\nＪＲ横浜線 淵野辺駅 北口 徒歩2分\n\n★☆定番人気のスタン...,"3,637円～11,819円/人（消費税込4,000円～13,000円/人）","3,637円～11,819円/人","4,000円～13,000円/人"
84,ホテルリブマックスＢＵＤＧＥＴ相模原,神奈川県 大和・相模原・町田西部,"3,091円～（消費税込3,400円～）",【10時チェックアウト】『リブ得 』ホテルリブマックス最安値のスペシャルプラン☆淵野辺駅北口...,～最寄駅からホテルまで～\nＪＲ横浜線 淵野辺駅 北口 徒歩2分\n\n★☆『リブ得 』ホテ...,"3,091円～10,046円/人（消費税込3,400円～11,050円/人）","3,091円～10,046円/人","3,400円～11,050円/人"
